In [1]:
import pandas as pd
import numpy as np
import re
from datetime import date, timedelta
import konlpy as kn
import MeCab as mc
import ekonlpy as ek
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [2]:
day = date(2024, 3, 1)
day_dateoffset = day - pd.DateOffset(months=1)
print(day_dateoffset)

day = date(2024, 3, 1)
day_timedelta = day - timedelta(days=31)
print(day_timedelta)

2024-02-01 00:00:00
2024-01-30


In [3]:
f = open('./crawl_result/call_rate.csv', 'r', encoding='utf-8')

# 데이터를 완전히 동일하게 가져올 수 있는가
headers = f.readline().rstrip() # 헤더 읽기, 맨 끝 줄바꿈기호 삭제
print(headers.split('\t'))
headers = headers.split('\t') # 같은 이름의 변수에 리스트로 바꾼 것 넣어주기

# 데이터 읽기
lines = f.readlines()
print(lines[:5])

new_datas = []
for line in lines: # 한 줄씩 
    data = line.rstrip().split('\t') # 줄바꿈 기호 지우고, ,로 잘라준다.
    dict_data = { header: data[i] for i, header in enumerate(headers) }
    new_datas.append(dict_data) # 데이터를 딕셔너리로 만들어서 넣어야된다.

f.close()

['date', 'price', 'label']
['2024-03-21\t3.5290\t-0.11\n', '2024-03-20\t3.5330\t0.56\n', '2024-03-19\t3.5130\t2.09\n', '2024-03-18\t3.4410\t-0.66\n', '2024-03-15\t3.4640\t-0.63\n']


In [4]:
def find_nearest_date(df, target_date):
    delta = pd.Timedelta(days=1)
    nearest_date = None,
    min_difference = pd.Timedelta.max

    for date in df['date']:
        difference = abs(target_date - date)
        if difference < min_difference:
            min_difference = difference
            nearest_date = date

    return nearest_date

def label_call(df):
    df_ud = pd.DataFrame(index=df.index, columns=['date', 'price', 'label'])

    # 'date' 열을 datetime 형식으로 변환
    df['date'] = pd.to_datetime(df['date'])

    for i, row in df.iterrows():
        time = row['date']
        time2 = time - pd.DateOffset(months=1)

        # 이전 달의 날짜가 인덱스에 있는지 확인
        if time2 in df['date'].values:
            prev_row = df[df['date'] == time2].iloc[0]
        else:
            # 이전 날짜가 없는 경우 가장 가까운 날짜로 설정
            nearest_date = find_nearest_date(df, time2)
            prev_row = df[df['date'] == nearest_date].iloc[0]

        if row['price'] > prev_row['price']:
            label = '1'
        elif row['price'] == prev_row['price']:
            label = 0
        else:
            label = '-1'
            
        df_ud.loc[i] = [time.date(), row['price'], label]

    return df_ud 

In [5]:
df = pd.DataFrame(new_datas)

call = label_call(df)
call

,date,price,label
0,2024-03-21,3.5290,-1
1,2024-03-20,3.5330,-1
2,2024-03-19,3.5130,-1
3,2024-03-18,3.4410,-1
4,2024-03-15,3.4640,-1
...,...,...,...
3253,2011-01-07,2.4900,0
3254,2011-01-06,2.4900,0
3255,2011-01-05,2.4900,0
3256,2011-01-04,2.5000,1


In [6]:
# 문장 분할을 위한 정규표현식 패턴
sentence_pattern = r'(?<=[.!?]) +'

# 데이터프레임으로부터 문서를 읽어와 딕셔너리 구조로 저장하는 함수
def read_documents_to_dict(df):
    date_dict = {}
    change_pattern = '(?<=[가-힣])\\.'
    news_pattern = ['\\[.*\\]', '\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?', '....기자']

    # DataFrame의 각 행에 대해 반복
    for index, row in df.iterrows():
        date = str(row['date']).split(' ')[0] 
        text = row['content'] 
        
        # 뉴스 패턴 제거
        news_text = text
        for pattern in news_pattern:
            news_text = re.sub(pattern, '', news_text)

        # 변경 패턴 적용
        change_text = re.sub(change_pattern, '@@@', news_text)  
        split_text = change_text.split('@@@')  
        split_text = [sent.strip() for sent in split_text]
        
        # 문서별로 문장 추가
        if date not in date_dict:
            date_dict[date] = []
        
        # 문장 추가
        if split_text:  # 빈 리스트인 경우 추가하지 않음
            date_dict[date].append(split_text)

    return date_dict

# 문서 데이터프레임 읽기
# -------------------------------------------------------------------------------------예시 경로 설정 필요 (여기만 건드시면 됩니다 함수 안건드셔도되요)-------------------------------------
jong_path = "./crawl_result/joongamg_news.json"
edaily_path = "./crawl_result/edaily_news.json"
money_path = "./crawl_result/moneytoday_news.json"
bone_path = "./crawl_result/bone_report.json"
min_path = "./crawl_result/minutes_re.tsv"

df_news_1 = pd.read_json(jong_path)
df_news_2 = pd.read_json(edaily_path)
df_news_3 = pd.read_json(money_path)
df_bone = pd.read_json(bone_path)
df_min = pd.read_csv(min_path, sep='\t')

# 문서들을 딕셔너리 구조로 저장
date_dict_1 = read_documents_to_dict(df_news_1)
date_dict_2 = read_documents_to_dict(df_news_2)
date_dict_3 = read_documents_to_dict(df_news_3)
date_dict_4 = read_documents_to_dict(df_bone)
date_dict_5 = read_documents_to_dict(df_min)

In [7]:
# 두 개의 딕셔너리를 합치는 함수
def merge_dicts(dict1, dict2):
    merged_dict = dict1.copy()
    for key, value in dict2.items():
        if key in merged_dict:
            merged_dict[key].extend(value)  # 이미 있는 키의 경우 값들을 합침
        else:
            merged_dict[key] = value  # 새로운 키의 경우 값을 그대로 추가
    return merged_dict



# 두 개의 딕셔너리 합치기 -------------------------- 함수 건들지 마시고 밑에꺼에 dict 값 넣으시면 됩니다. ------------------------------------------------
merged_date_dict_1 = merge_dicts(date_dict_1, date_dict_2)
merged_date_dict_2 = merge_dicts(date_dict_3, date_dict_4)
merged_date_dict_3 = merge_dicts(merged_date_dict_1, merged_date_dict_2)

final_dict = merge_dicts(merged_date_dict_3, date_dict_5)

In [8]:
final_dict['2013-03-14'][0][0]
####  딕셔너리 이름 [날짜] [몇번째 문서인지] [그 문서의 몇번째 문장인지]

'“북한에서 내전이 벌어진다'

In [9]:
# # 토큰화 및 n-gram 생성 함수 정의
# def tokenize_with_ngrams(sentence, n):
#     tokens_with_pos = tokenizer.pos(sentence)
#     tokens = ['{}/{}'.format(token, pos) for token, pos in tokens_with_pos if pos in ['VA', 'VX', 'VV', 'NNG', 'MAG']]
#     ngrams = generate_ngrams(tokens, n)
#     return [tuple(ngram.split()) for ngram in ngrams]

# # 토큰화된 결과를 데이터프레임으로 출력
# token_data = {'date': [], 'document': [], 'tokens': []}
# n = 1  # n-gram의 n값 설정
# # count = 0  # 출력된 결과의 수를 세는 변수

# for date, documents in final_dict.items():
#     for doc_num, sentences in enumerate(documents, start=1):
#         # print(f"{date}의 문서 {doc_num} 토큰화 및 {n}-gram 생성 결과:")
#         for sentence in sentences:
#             ngram_tokens = tokenize_with_ngrams(sentence, n)
#             token_data['date'].append(date)
#             token_data['document'].append(f"문서 {doc_num}")
#             token_data['tokens'].append(ngram_tokens)
#     #         count += 1
#     #         if count >= 10:  # 처음 10개의 결과만 확인하고 루프 종료
#     #             break
#     #     else:
#     #         continue
#     #     break
#     # else:
#     #     continue
#     # break

# df = pd.DataFrame(token_data)
# df = df[df['tokens'].apply(lambda x: len(x) > 0)]

In [9]:
# 특정 품사만 토큰 
from ekonlpy.tag import Mecab

# ekonlpy의 Mecab 형태소 분석기 인스턴스 생성
tokenizer = Mecab()

# 토큰화 함수 정의
def tokenize(sentence):
    tokens_with_pos = tokenizer.pos(sentence)
    tokens = [(token, pos) for token, pos in tokens_with_pos if pos in ['VA', 'VX', 'VV', 'NNG', 'MAG' ]]
    return tokens

# 토큰화된 결과를 데이터프레임으로 출력
token_data = {'date': [], 'document': [], 'tokens': []}

for date, documents in final_dict.items():
    for doc_num, sentences in enumerate(documents, start=1):
        # print(f"{date}의 문서 {doc_num} 토큰화 결과:")
        for sentence in sentences:
            tokens = tokenize(sentence)
            token_data['date'].append(date)
            token_data['document'].append(f"문서 {doc_num}")
            token_data['tokens'].append(tokens)

df = pd.DataFrame(token_data)

In [10]:
# 'tokens' 열이 '[]'인 행 제거
df = df[df['tokens'].apply(len) > 0]

In [11]:
# 'date' 열을 datetime으로 변환
call['date'] = pd.to_datetime(call['date'])
df['date'] = pd.to_datetime(df['date'])

# 'date' 열을 기준으로 외부 조인 수행
merged_df = pd.merge(call, df, on='date', how='outer')

# 'key' 칼럼을 인덱스로 설정
merged_df.set_index('date', inplace=True)

# print(merged_df)

In [12]:
# 'label' 칼럼의 값이 '1(매파)'인 행과 '-1(비둘기파)'인 행으로 데이터프레임 분리
df_P = merged_df[merged_df['label'] == '1']
df_N = merged_df[merged_df['label'] == '-1']

# Nan인 행 제거
df_P = df_P.dropna()
df_N = df_N.dropna()

# 필요없는 열 제거 
df_P = df_P.drop(['price'], axis=1) # 'date'
df_N = df_N.drop(['price'], axis=1) # 'date'

# word_df = pd.concat([df_P,df_N],axis=0)
# x_tain = word_df["words"]
# y_tain = word_df["label"]

In [13]:
df_P = df_P.rename(columns={'label': 'P'})
# df_P
# df_P.to_csv("df_P_1gram.csv", mode='w')

In [14]:
df_N = df_N.rename(columns={'label': 'N'})
# df_N
# df_N.to_csv("df_N_1gram.csv", mode='w')

#### ------------------------------------------------------------------------------------------------------

In [49]:
# P 1_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_P['tokens'] = df_P['tokens'].apply(lambda x: [t for t in x if t])

n = 1  # 1-gram 설정
ngrams = generate_ngrams(df_P['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 10 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 15}

# 데이터프레임으로 변환
P_1gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
P_1gram_df['P'] = 1
P_1gram_df

,ngram,frequency,P
0,대통령실/NNG,707,1
1,대변인/NNG,322,1
2,정부/NNG,16931,1
3,제2금융권/NNG,527,1
4,고금리/NNG,4659,1
...,...,...,...
14649,생애최초주택구입자금/NNG,25,1
14650,방사능유출/NNG,17,1
14651,다이렉트뱅킹/NNG,25,1
14652,퇴석/NNG,16,1


In [41]:
# N 1_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_N['tokens'] = df_N['tokens'].apply(lambda x: [t for t in x if t])

n = 1  # 1-gram 설정
ngrams = generate_ngrams(df_N['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 10 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 10}

# 데이터프레임으로 변환
N_1gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
N_1gram_df['N'] = -1
N_1gram_df

,ngram,frequency,N
0,지난/VV,15873,-1
1,일/NNG,47516,-1
2,오후/NNG,1860,-1
3,업/NNG,643,-1
4,비트/NNG,52,-1
...,...,...,...
16646,온렌딩/NNG,13,-1
16647,퇴석/NNG,12,-1
16648,무상지분율/NNG,10,-1
16649,중심선/NNG,21,-1


In [50]:
P_1gram_df['frequency'].sum()

6066674

In [42]:
N_1gram_df['frequency'].sum()

5353375

In [51]:
merged_df = pd.merge(P_1gram_df , N_1gram_df , on='ngram', how='inner')
merged_df = merged_df.drop(['P', 'N'], axis=1) 
merged_df = merged_df.rename(columns={'frequency_x': 'P_count'})
merged_df = merged_df.rename(columns={'frequency_y': 'N_count'})
merged_df.set_index('ngram', inplace=True)
merged_df

,P_count,N_count
ngram,,
대통령실/NNG,707,137
대변인/NNG,322,161
정부/NNG,16931,14319
제2금융권/NNG,527,625
고금리/NNG,4659,3517
...,...,...
선물사/NNG,17,29
리시브/NNG,15,36
생애최초주택구입자금/NNG,25,58


In [52]:
merged_df.to_csv("count_1gram.csv", mode='w')

#### ----------------------------------------------------------- 2_gram ------------------------------------------------------------------

In [56]:
# P 2_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_P['tokens'] = df_P['tokens'].apply(lambda x: [t for t in x if t])

n = 2  # 2-gram 설정
ngrams = generate_ngrams(df_P['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 15 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 15}

# 데이터프레임으로 변환
P_2gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
P_2gram_df['P'] = 1
P_2gram_df

,ngram,frequency,P
0,대통령실/NNG 대변인/NNG,16,1
1,고금리/NNG 대출/NNG,479,1
2,대출/NNG 받/VV,2282,1
3,받/VV 소상공인/NNG,29,1
4,소상공인/NNG 이자/NNG,26,1
...,...,...,...
48512,전강후약/NNG 전망/NNG,22,1
48513,인한/VV 전강후약/NNG,20,1
48514,스프레드/NNG 과도/NNG,15,1
48515,적절/NNG 보입니다/VV,20,1


In [60]:
# N 2_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_N['tokens'] = df_N['tokens'].apply(lambda x: [t for t in x if t])

n = 2  # 1-gram 설정
ngrams = generate_ngrams(df_N['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 8 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 8}

# 데이터프레임으로 변환
N_2gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
N_2gram_df['N'] = -1
N_2gram_df

,ngram,frequency,N
0,지난/VV 일/NNG,5715,-1
1,일/NNG 오후/NNG,607,-1
2,업/NNG 비트/NNG,34,-1
3,비트코인/NNG 시세/NNG,12,-1
4,나오/VV 있/VX,687,-1
...,...,...,...
87120,매수전략/NNG 제안/NNG,8,-1
87121,축소/NNG 베팅/NNG,8,-1
87122,금통위/NNG 앞둔/VV,17,-1
87123,통화정책/NNG 경계감/NNG,12,-1


In [58]:
P_2gram_df['frequency'].sum()

2770148

In [61]:
N_2gram_df['frequency'].sum()

2785864

In [62]:
merged_df = pd.merge(P_2gram_df , N_2gram_df , on='ngram', how='inner')
merged_df = merged_df.drop(['P', 'N'], axis=1) 
merged_df = merged_df.rename(columns={'frequency_x': 'P_count'})
merged_df = merged_df.rename(columns={'frequency_y': 'N_count'})
merged_df.set_index('ngram', inplace=True)
merged_df

,P_count,N_count
ngram,,
고금리/NNG 대출/NNG,479,492
대출/NNG 받/VV,2282,2180
받/VV 소상공인/NNG,29,22
이자/NNG 환급/NNG,47,29
브리핑/NNG 통해/VV,44,32
...,...,...
보이/VV 목표/NNG,86,14
앞둔/VV 통화정책/NNG,22,11
목표/NNG 매수전략/NNG,78,8


In [63]:
merged_df.to_csv("count_2gram.csv", mode='w')

#### ----------------------------------------------------------- 3_gram ------------------------------------------------------------------

In [69]:
# P 3_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_P['tokens'] = df_P['tokens'].apply(lambda x: [t for t in x if t])

n = 3  # 2-gram 설정
ngrams = generate_ngrams(df_P['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 15 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 10}

# 데이터프레임으로 변환
P_3gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
P_3gram_df['P'] = 1
P_3gram_df

,ngram,frequency,P
0,고금리/NNG 대출/NNG 받/VV,73,1
1,대출/NNG 받/VV 소상공인/NNG,17,1
2,통해/VV 오/VV 일/NNG,10,1
3,은행권/NNG 민생/NNG 금융지원/NNG,13,1
4,민생/NNG 금융지원/NNG 방안/NNG,17,1
...,...,...,...
30538,약세/NNG 가격/NNG 부담/NNG,10,1
30539,금일/NNG 채권시장/NNG 미국/NNG,19,1
30540,매수/NNG 시점/NNG 전일/NNG,16,1
30541,시점/NNG 전일/NNG 종가/NNG,27,1


In [65]:
# N 3_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_N['tokens'] = df_N['tokens'].apply(lambda x: [t for t in x if t])

n = 3  # 1-gram 설정
ngrams = generate_ngrams(df_N['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 8 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 8}

# 데이터프레임으로 변환
N_3gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
N_3gram_df['N'] = -1
N_3gram_df

,ngram,frequency,N
0,지난/VV 일/NNG 오후/NNG,63,-1
1,일/NNG 현지/NNG 시간/NNG,4909,-1
2,현지/NNG 시간/NNG 미/NNG,98,-1
3,시간/NNG 미/NNG 연방준비제도/NNG,10,-1
4,미/NNG 연방준비제도/NNG Fed/NNG,574,-1
...,...,...,...
35359,국고채/NNG 금리/NNG 목표/NNG,10,-1
35360,부담/NNG 인해/VV 약세/NNG,9,-1
35361,인해/VV 약세/NNG 진행/NNG,12,-1
35362,약세/NNG 진행/NNG 전망/NNG,11,-1


In [70]:
P_3gram_df['frequency'].sum()

828443

In [68]:
N_3gram_df['frequency'].sum()

743666

In [80]:
merged_df = pd.merge(P_3gram_df , N_3gram_df , on='ngram', how='inner')
merged_df = merged_df.drop(['P', 'N'], axis=1) 
merged_df = merged_df.rename(columns={'frequency_x': 'P_count'})
merged_df = merged_df.rename(columns={'frequency_y': 'N_count'})
merged_df.set_index('ngram', inplace=True)
merged_df

,P_count,N_count
ngram,,
고금리/NNG 대출/NNG 받/VV,73,63
대출/NNG 받/VV 소상공인/NNG,17,8
통해/VV 오/VV 일/NNG,10,11
금융지원/NNG 방안/NNG 마련/NNG,10,8
이상/NNG 고금리/NNG 대출/NNG,42,42
...,...,...
금일/NNG 전략/NNG 국고채/NNG,71,10
전략/NNG 국고채/NNG 금리/NNG,71,10
국고채/NNG 금리/NNG 목표/NNG,71,10


In [81]:
merged_df.to_csv("count_3gram.csv", mode='w')

#### ----------------------------------------------------------- 4_gram ------------------------------------------------------------------

In [73]:
# P 4_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_P['tokens'] = df_P['tokens'].apply(lambda x: [t for t in x if t])

n = 4  # 2-gram 설정
ngrams = generate_ngrams(df_P['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 15 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 10}

# 데이터프레임으로 변환
P_4gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
P_4gram_df['P'] = 1
P_4gram_df

,ngram,frequency,P
0,지난해/NNG 분기/NNG 말/NNG 기준/NNG,22,1
1,총부채/NNG 원리금/NNG 상환/NNG 비율/NNG,399,1
2,예금은행/NNG 신규/NNG 취급액/NNG 기준/NNG,57,1
3,변동/NNG 금리/NNG 주담대/NNG 받/VV,15,1
4,파생/NNG 결합/NNG 펀드/NNG 사태/NNG,17,1
...,...,...,...
10942,채권시장/NNG 약세/NNG 가격/NNG 부담/NNG,10,1
10943,금일/NNG 채권시장/NNG 미국/NNG 채권시장/NNG,19,1
10944,따른/VV 세계/NNG 경기/NNG 불확실성/NNG,12,1
10945,매수/NNG 시점/NNG 전일/NNG 종가/NNG,15,1


In [74]:
# N 4_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_N['tokens'] = df_N['tokens'].apply(lambda x: [t for t in x if t])

n = 4  # 1-gram 설정
ngrams = generate_ngrams(df_N['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 8 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 8}

# 데이터프레임으로 변환
N_4gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
N_4gram_df['N'] = -1
N_4gram_df

,ngram,frequency,N
0,일/NNG 현지/NNG 시간/NNG 미/NNG,97,-1
1,현지/NNG 시간/NNG 미/NNG 연방준비제도/NNG,10,-1
2,시간/NNG 미/NNG 연방준비제도/NNG Fed/NNG,9,-1
3,미/NNG 연방준비제도/NNG Fed/NNG 연준/NNG,135,-1
4,연방준비제도/NNG Fed/NNG 연준/NNG 연내/NNG,11,-1
...,...,...,...
12709,국고채/NNG 금리/NNG 목표/NNG 매도전략/NNG,8,-1
12710,채권시장/NNG 미국/NNG 채권시장/NNG 약세/NNG,10,-1
12711,부담/NNG 인해/VV 약세/NNG 진행/NNG,9,-1
12712,인해/VV 약세/NNG 진행/NNG 전망/NNG,10,-1


In [75]:
P_4gram_df['frequency'].sum()

248047

In [76]:
N_4gram_df['frequency'].sum()

223873

In [82]:
merged_df = pd.merge(P_4gram_df , N_4gram_df , on='ngram', how='inner')
merged_df = merged_df.drop(['P', 'N'], axis=1) 
merged_df = merged_df.rename(columns={'frequency_x': 'P_count'})
merged_df = merged_df.rename(columns={'frequency_y': 'N_count'})
merged_df.set_index('ngram', inplace=True)
merged_df

,P_count,N_count
ngram,,
지난해/NNG 분기/NNG 말/NNG 기준/NNG,22,23
총부채/NNG 원리금/NNG 상환/NNG 비율/NNG,399,188
예금은행/NNG 신규/NNG 취급액/NNG 기준/NNG,57,25
파생/NNG 결합/NNG 펀드/NNG 사태/NNG,17,31
일/NNG 국내/NNG 국고채/NNG 시장/NNG,78,88
...,...,...
채권시장/NNG 미국/NNG 채권시장/NNG 약세/NNG,64,10
금일/NNG 전략/NNG 국고채/NNG 금리/NNG,71,10
전략/NNG 국고채/NNG 금리/NNG 목표/NNG,71,10


In [83]:
merged_df.to_csv("count_4gram.csv", mode='w')

#### ----------------------------------------------------------- 5_gram ------------------------------------------------------------------

In [89]:
# P 5_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_P['tokens'] = df_P['tokens'].apply(lambda x: [t for t in x if t])

n = 5  # 5-gram 설정
ngrams = generate_ngrams(df_P['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 15 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 8}

# 데이터프레임으로 변환
P_5gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
P_5gram_df['P'] = 1
P_5gram_df

,ngram,frequency,P
0,고정/NNG 금리/NNG 후/NNG 변동/NNG 금리/NNG,8,1
1,페드워치/NNG 따르/VV 연방기금/NNG 금리선물/NNG 시장/NNG,24,1
2,마켓/NNG 포인트/NNG 따르/VV 지난/VV 일/NNG,16,1
3,수정/NNG 경제/NNG 전망/NNG 통해/VV 올해/NNG,23,1
4,연방준비제도/NNG Fed/NNG 연준/NNG 기준금리/NNG 인하/NNG,12,1
...,...,...,...
7931,약세/NNG 가격/NNG 부담/NNG 인해/VV 약세/NNG,8,1
7932,금일/NNG 채권시장/NNG 미국/NNG 채권시장/NNG 약세/NNG,11,1
7933,금일/NNG 채권시장/NNG 미국/NNG 채권시장/NNG 강세/NNG,8,1
7934,매수/NNG 시점/NNG 전일/NNG 종가/NNG 수준/NNG,15,1


In [90]:
# N 5_gram
from collections import defaultdict
import pandas as pd

# generate_ngrams 함수 정의
def generate_ngrams(text, n):
    ngrams = []
    for row in text:
        # 각 행의 토큰들을 n-gram으로 변환
        for i in range(len(row)-n+1):
            ngrams.append(' '.join([f"{token[0]}/{token[1]}" for token in row[i:i+n]]))
    return ngrams

# count_ngrams 함수 정의
def count_ngrams(ngrams):
    ngram_counts = defaultdict(int)
    for ngram in ngrams:
        ngram_counts[ngram] += 1
    return ngram_counts

# 주어진 데이터프레임에서 빈 튜플을 제거하고 각 튜플의 토큰을 리스트로 변환
df_N['tokens'] = df_N['tokens'].apply(lambda x: [t for t in x if t])

n = 5  # 5-gram 설정
ngrams = generate_ngrams(df_N['tokens'], n)
ngram_counts = count_ngrams(ngrams)

# 빈도수가 8 이상인 n-gram만 선택
selected_ngrams = {ngram: count for ngram, count in ngram_counts.items() if count >= 7}

# 데이터프레임으로 변환
N_5gram_df = pd.DataFrame(list(selected_ngrams.items()), columns=['ngram', 'frequency'])
N_5gram_df['N'] = -1
N_5gram_df

,ngram,frequency,N
0,일/NNG 현지/NNG 시간/NNG 미/NNG 연방준비제도/NNG,10,-1
1,현지/NNG 시간/NNG 미/NNG 연방준비제도/NNG Fed/NNG,9,-1
2,미국/NNG 연방준비제도/NNG Fed/NNG 연준/NNG 의장/NNG,56,-1
3,일/NNG 현지/NNG 시간/NNG 연방준비제도/NNG Fed/NNG,10,-1
4,다우존스30산업평균지수/NNG 전장/NNG 포인트/NNG 오른/VV 거래/NNG,7,-1
...,...,...,...
7839,금일/NNG 전략/NNG 국고채/NNG 금리/NNG 목표/NNG,10,-1
7840,전략/NNG 국고채/NNG 금리/NNG 목표/NNG 매도전략/NNG,8,-1
7841,국고채/NNG 금리/NNG 목표/NNG 매도전략/NNG 제안/NNG,8,-1
7842,부담/NNG 인해/VV 약세/NNG 진행/NNG 전망/NNG,9,-1


In [91]:
P_5gram_df['frequency'].sum()

134665

In [92]:
N_5gram_df['frequency'].sum()

114698

In [93]:
merged_df = pd.merge(P_5gram_df , N_5gram_df , on='ngram', how='inner')
merged_df = merged_df.drop(['P', 'N'], axis=1) 
merged_df = merged_df.rename(columns={'frequency_x': 'P_count'})
merged_df = merged_df.rename(columns={'frequency_y': 'N_count'})
merged_df.set_index('ngram', inplace=True)
merged_df

,P_count,N_count
ngram,,
고정/NNG 금리/NNG 후/NNG 변동/NNG 금리/NNG,8,13
페드워치/NNG 따르/VV 연방기금/NNG 금리선물/NNG 시장/NNG,24,18
마켓/NNG 포인트/NNG 따르/VV 지난/VV 일/NNG,16,20
수정/NNG 경제/NNG 전망/NNG 통해/VV 올해/NNG,23,20
채권/NNG 금리/NNG 가격/NNG 반대/NNG 움직인다/VV,76,75
...,...,...
당일/NNG 채권시장/NNG 미국/NNG 채권시장/NNG 강세/NNG,39,9
금일/NNG 전략/NNG 국고채/NNG 금리/NNG 목표/NNG,71,10
부담/NNG 인해/VV 약세/NNG 진행/NNG 전망/NNG,14,9


In [94]:
merged_df.to_csv("count_5gram.csv", mode='w')